In [47]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import os
import time
from datetime import datetime

In [48]:
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))
PLUGINS_DIR = os.path.join(BASE_DIR, 'plugins')
WEBDRIVERS_DIR = os.path.join(PLUGINS_DIR, 'webdrivers')

In [59]:
class Spider():
    def __init__(self, navegador, url=''):
        self._navegador = navegador
        self._url = url

        if self._navegador == 'edge':
            # options = webdriver.EdgeOptions()
            # options.add_argument("--headless")

            driver_exe = os.path.join(WEBDRIVERS_DIR, 'msedgedriver.exe')            
            # self._driver = webdriver.Edge(service=Service(executable_path=driver_exe), options=options)
            self._driver = webdriver.Edge(service=Service(executable_path=driver_exe))
        else:
            print('navegador não implementado!')

    def abrir_site(self, url=''):
        if url:
            self._url = url

        if not self._url:            
            raise ValueError('URL não informada!')        

        self._driver.get(self._url)
        time.sleep(2)    

    def close(self):
        self._driver.close()

    def quit(self):
        self._driver.quit()

In [57]:
class Decolar(Spider):
    def __init__(self, navegador, url=''):
        super().__init__(navegador, url)

        self.SITE_MAP = {
            'buttons': {
                'buscar': {
                    'xpath': '//*[@id="searchbox-sbox-box-packages"]/div/div/div/div/div[3]/div[4]/button'
                },
                'cookie': {
                    'xpath': '//*[@id="lgpd-banner"]/div/a[2]'
                },
                'aplicar_calendar': {
                    'xpath':'//*[@id="component-modals"]/div[1]/div[2]/div[1]/button'
                }
            },
            'inputs': {
                'origem': {
                    'xpath': '//*[@id="searchbox-sbox-box-packages"]/div/div/div/div/div[3]/div[1]/div/div[1]/div[1]/div/input'
                },
                'destino': {
                    'xpath': '//*[@id="searchbox-sbox-box-packages"]/div/div/div/div/div[3]/div[1]/div/div[2]/div/div/input'
                },
                'dtini':{
                    'xpath_click': '//*[@id="searchbox-sbox-box-packages"]/div/div/div/div/div[3]/div[2]/div[1]/div[1]/div/div/div/div/input'
                },
                'dtfin':{
                    'xpath_click': '//*[@id="searchbox-sbox-box-packages"]/div/div/div/div/div[3]/div[2]/div[1]/div[2]/div/div/div/div/input'
                },
                'calendar_container': {  
                    'xpath_month_year1': '//*[@id="component-modals"]/div[1]/div[1]/div[2]/div[1]',                    
                    'xpath_month_year2': '//*[@id="component-modals"]/div[1]/div[1]/div[2]/div[2]',                    
                    'xpath_button_left': '//*[@id="component-modals"]/div[1]/div[1]/div[2]/a[1]',
                    'xpath_button_right': '//*[@id="component-modals"]/div[1]/div[1]/div[2]/a[2]',                    
                },                
            }
            
        }

        self._actions = ActionChains(self._driver)

    def _get_year_month(self):
        year1, month1 = [int(n) for n in self._driver.find_element(By.XPATH, self.SITE_MAP['inputs']['calendar_container']['xpath_month_year1']).get_attribute('data-month').split('-')]
        year2, month2 = [int(n) for n in self._driver.find_element(By.XPATH, self.SITE_MAP['inputs']['calendar_container']['xpath_month_year2']).get_attribute('data-month').split('-')]

        return year1, month1, year2, month2

    def _click_day_calendar(self, date):
        time.sleep(1)
        year1, month1, year2, month2 = self._get_year_month()        

        year_date = int(date.strftime('%Y'))        
        month_date = int(date.strftime('%m'))        
        day_date = int(date.strftime('%d'))


        while ((year1 != year_date) and (year2 != year_date)) or ((month1 != month_date) and (month2 != month_date)):
            year1, month1, year2, month2 = self._get_year_month()
            
            if (year1 and year2 < year_date) or (month1 and month2 < month_date):
                self._driver.find_element(By.XPATH, self.SITE_MAP['inputs']['calendar_container']['xpath_button_right']).click()
            elif (year1 and year2 > year_date) or (month1 and month2 > month_date):
                self._driver.find_element(By.XPATH, self.SITE_MAP['inputs']['month_date']['xpath_button_left']).click()         
        
        if month1 == month_date:
            month_grid = self._driver.find_element(By.XPATH, self.SITE_MAP['inputs']['calendar_container']['xpath_month_year1']).find_element(By.CLASS_NAME, 'sbox5-monthgrid-dates')            
        else:
            month_grid = self._driver.find_element(By.XPATH, self.SITE_MAP['inputs']['calendar_container']['xpath_month_year2']).find_element(By.CLASS_NAME, 'sbox5-monthgrid-dates')            
        
        days = month_grid.find_elements(By.CLASS_NAME, 'sbox5-monthgrid-datenumber')     

        for day in days:   
            if day.text[:2].isdigit():
                day_number = int(day.text[:2])
            else:
                day_number = int(day.text[0])

            if day_date == day_number:
                if '-disabled' in day.get_attribute('class'):
                    raise Exception('Data indisponível. Impossível selecionar!')

                day.click()
                time.sleep(1)
                break                                   

    
    def pesquisar_voo(self, origem, destino, date_defined, dtini='', dtfin=''):
        # month_list = ['JANEIRO', 'FEVEREIRO', 'MARÇO', 'ABRIL', 'MAIO', 'JUNHO', 'JULHO', 'AGOSTO', 'SETEMBRO', 'OUTUBRO', 'NOVEMBRO', 'DEZEMBRO']

        if date_defined:
            dtini = datetime.strptime(dtini, '%d/%m/%Y').date()
            dtfin = datetime.strptime(dtfin, '%d/%m/%Y').date()        

        cookie_button = self._driver.find_element(By.XPATH, self.SITE_MAP['buttons']['cookie']['xpath'])
        if cookie_button:
            cookie_button.click()

        origem_element = self._driver.find_element(By.XPATH, self.SITE_MAP['inputs']['origem']['xpath'])
        self._actions.move_to_element(origem_element).perform()
        origem_element.click()
        time.sleep(1)      
        origem_element.send_keys(origem)  
        time.sleep(1)  
        origem_element.send_keys(Keys.ENTER)

        destino_element = self._driver.find_element(By.XPATH, self.SITE_MAP['inputs']['destino']['xpath'])
        self._actions.move_to_element(destino_element).perform()
        destino_element.click()
        time.sleep(1)      
        destino_element.send_keys(destino)    
        time.sleep(1)  
        destino_element.send_keys(Keys.ENTER)        

        dtini_element = self._driver.find_element(By.XPATH, self.SITE_MAP['inputs']['dtini']['xpath_click'])
        self._actions.move_to_element(dtini_element).perform()
        dtini_element.click()       
       
        self._click_day_calendar(dtini)
        self._click_day_calendar(dtfin)     
        
        calendar_aplicar_element = self._driver.find_element(By.XPATH, self.SITE_MAP['buttons']['aplicar_calendar']['xpath'])
        self._actions.move_to_element(calendar_aplicar_element).perform()
        calendar_aplicar_element.click()        
                        
        buscar_element = self._driver.find_element('xpath', self.SITE_MAP['buttons']['buscar']['xpath'])
        self._actions.move_to_element(buscar_element).perform()
        time.sleep(1)
        buscar_element.click()

In [62]:
try:
    decolar = Decolar('edge')
    decolar.abrir_site('https://www.decolar.com/pacotes/')
    decolar.pesquisar_voo('São Paulo', 'Tóquio', True, '06/08/2023', '22/08/2023')
finally:
    pass
    # decolar._driver.close()